In [1]:
import numpy as np
from gammapy.datasets import Datasets
from gammapy.modeling.models import *
from gammapy.modeling import Fit
from gammapy.data import GTI
from gammapy.utils.pbar import progress_bar
from gammapy.maps import TimeMapAxis
from astropy.table import Table
import astropy.units as u

In [17]:
class TimeResolvedSpectroscopyEstimator():
    def __init__(self, model, time_intervals=None, fit=None, atol="1e-6 s"):
        self.model = model
        self.time_intervals = time_intervals
        self.atol = atol
        if fit is None:
            fit = Fit()

        self.fit = fit
        
    def run(self, datasets):
        
        
        models = Models()
        valid_intervals = []
        
        if not isinstance(datasets, Datasets):
            datasets = Datasets(datasets)
        if self.time_intervals is None:
            gti = datasets.gti
        else:
            gti = GTI.from_time_intervals(self.time_intervals)
            
        gti = gti.union(overlap_ok=False, merge_equal=False)
        
        index = 0
        for t_min, t_max in progress_bar(
            gti.time_intervals, desc="Time intervals selection"
        ):
        
            datasets_to_fit = datasets.select_time(
                time_min=t_min, time_max=t_max, atol=self.atol
            )
            if len(datasets_to_fit) == 0:
                log.info(
                    f"No Dataset for the time interval {t_min} to {t_max}. Skipping interval."
                )
                continue
            
            model = self.fit_in_bin(datasets_to_fit, index=index)
            models.append(model)
            valid_intervals.append([t_min, t_max])
            index = index +1
            
            
        gti = GTI.from_time_intervals(valid_intervals)
        
        self.models = models
        self._gti = gti
        self.axis = TimeMapAxis.from_gti(gti)
        
        return self.create_table()
    
    def create_table(self):
        col_names = []
        col_unit = []
        for par in self.model.parameters.free_parameters:
            col_names.append(par.name)
            col_names.append(par.name+"_err")
            unt = par.unit #TODO: Why does dimesionless unit not work directly?
            if unt is u.Unit():
                unt=""
            col_unit.append(unt)
            col_unit.append(unt)
    
    
        t = Table(names=col_names, units=col_unit)

        for i in range(self.axis.nbin):
            col_data = []
            for name in sky_model.parameters.free_parameters.names:
                col_data.append(self.models[i].parameters[name].value)
                col_data.append(self.models[i].parameters[name].error)
            t.add_row(col_data)
        
        t.add_columns(self._gti.table.columns, indexes=[0,0])
        return t
        
        
    
    def fit_in_bin(self, datasets_in_bin, index):
        model_in_bin = self.model.copy(name="Model_bin_" + str(index))
        datasets_in_bin.models = model_in_bin
        result = self.fit.run(datasets_in_bin)
        return model_in_bin
        
        

## Run the above estimator to get the model in each bin

In [18]:
import logging
import numpy as np
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time
from regions import CircleSkyRegion

# %matplotlib inline
import matplotlib.pyplot as plt

log = logging.getLogger(__name__)

from gammapy.data import DataStore
from gammapy.datasets import Datasets, SpectrumDataset
from gammapy.estimators import LightCurveEstimator
from gammapy.estimators.utils import get_rebinned_axis
from gammapy.makers import (
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
)
from gammapy.maps import MapAxis, RegionGeom
from gammapy.modeling.models import PowerLawSpectralModel, SkyModel

In [19]:
from gammapy.utils.check import check_tutorials_setup

check_tutorials_setup()


System:

	python_executable      : /home/usuario/anaconda3/envs/gammapy-dev/bin/python 
	python_version         : 3.11.10    
	machine                : x86_64     
	system                 : Linux      


Gammapy package:

	version                : 0.7.dev17998+gba580c792.d20240910 
	path                   : /home/usuario/Gammapy-dev/gammapy/gammapy 


Other packages:

	numpy                  : 2.1.1      
	scipy                  : 1.13.1     
	astropy                : 6.1.3      
	regions                : 0.9        
	click                  : 8.1.7      
	yaml                   : 6.0.2      
	IPython                : 8.27.0     
	jupyterlab             : 3.5.3      
	matplotlib             : 3.9.2      
	pandas                 : 2.2.2      
	healpy                 : 1.17.3     
	iminuit                : 2.29.1     
	sherpa                 : not installed 
	naima                  : 0.10.0     
	emcee                  : 3.1.6      
	corner                 : 2.2.2      
	ray             

In [20]:
data_store = DataStore.from_dir("/home/usuario/Gammapy-dev/gammapy-data/hess-dl3-dr1/")
target_position = SkyCoord(329.71693826 * u.deg, -30.2255890 * u.deg, frame="icrs")
selection = dict(
    type="sky_circle",
    frame="icrs",
    lon=target_position.ra,
    lat=target_position.dec,
    radius=2 * u.deg,
)
obs_ids = data_store.obs_table.select_observations(selection)["OBS_ID"]
observations = data_store.get_observations(obs_ids)
print(f"Number of selected observations : {len(observations)}")

Number of selected observations : 21


In [21]:
t0 = Time("2006-07-29T20:30")
duration = 15 * u.min
n_time_bins = 25
times = t0 + np.arange(n_time_bins) * duration
time_intervals = [Time([tstart, tstop]) for tstart, tstop in zip(times[:-1], times[1:])]
print(time_intervals[-1].mjd)

[53946.09375    53946.10416667]


In [22]:
short_observations = observations.select_time(time_intervals)
# check that observations have been filtered
print(f"Number of observations after time filtering: {len(short_observations)}\n")
print(short_observations[1].gti)

Number of observations after time filtering: 34

GTI info:
- Number of GTIs: 1
- Duration: 461.99999999999545 s
- Start: 207521165.184 s MET
- Start: 2006-07-29T20:45:00.000 (time standard: UTC)
- Stop: 207521627.184 s MET
- Stop: 2006-07-29T20:53:47.184 (time standard: TT)



In [23]:
# Target definition
energy_axis = MapAxis.from_energy_bounds("0.4 TeV", "20 TeV", nbin=10)
energy_axis_true = MapAxis.from_energy_bounds(
    "0.1 TeV", "40 TeV", nbin=20, name="energy_true"
)

on_region_radius = Angle("0.11 deg")
on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)

geom = RegionGeom.create(region=on_region, axes=[energy_axis])

In [24]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker()
safe_mask_masker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

In [25]:
datasets = Datasets()

dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

for obs in short_observations:
    dataset = dataset_maker.run(dataset_empty.copy(), obs)

    dataset_on_off = bkg_maker.run(dataset, obs)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, obs)
    datasets.append(dataset_on_off)

In [26]:
spectral_model = PowerLawSpectralModel(
    index=3.4, amplitude=2e-11 * u.Unit("1 / (cm2 s TeV)"), reference=1 * u.TeV
)
spectral_model.parameters["index"].frozen = False

sky_model = SkyModel(spatial_model=None, spectral_model=spectral_model, name="pks2155")

### Run estimator

In [27]:
est = TimeResolvedSpectroscopyEstimator(time_intervals=time_intervals, model=sky_model)

`est.run` will return an Astropy Table which can be used for further plotting etc

In [28]:
est.run(datasets)

START,STOP,index,index_err,amplitude,amplitude_err
,,,,1 / (TeV s cm2),1 / (TeV s cm2)
Time,Time,float64,float64,float64,float64
2006-07-29T20:30:00.000,2006-07-29T20:45:00.000,4.010482311802338,0.3537158056450928,1.0214504285333425e-10,1.29186358070327e-11
2006-07-29T20:45:00.000,2006-07-29T21:00:00.000,4.124876431213361,0.22570703650238536,1.2505685602677126e-10,1.1393229814393363e-11
2006-07-29T21:00:00.000,2006-07-29T21:15:00.000,3.588524935692903,0.13050373870495793,1.6486841145256898e-10,9.821712255009192e-12
2006-07-29T21:15:00.000,2006-07-29T21:30:00.000,3.4272779026627163,0.10213616919792164,1.6401318921602673e-10,1.0033774310640316e-11
2006-07-29T21:30:00.000,2006-07-29T21:45:00.000,3.4883166144304676,0.07291062006692837,2.1617552129101694e-10,1.085919833146256e-11
2006-07-29T21:45:00.000,2006-07-29T22:00:00.000,3.683785331767052,0.08922938199279859,1.8441229219329163e-10,1.1660257334679285e-11
2006-07-29T22:00:00.000,2006-07-29T22:15:00.000,3.549614139050003,0.08729158692106088,1.5876816221161472e-10,9.684142861230883e-12
2006-07-29T22:15:00.000,2006-07-29T22:30:00.000,3.68523317621821,0.11537944789849014,1.1147774366626535e-10,9.201015647599153e-12


The fitted model in each bin stored in `est.models`. 
Serialisation to be added...

In [29]:
est.models

In [164]:

col_names = []
col_unit = []
for par in sky_model.parameters.free_parameters:
    col_names.append(par.name)
    col_names.append(par.name+"_err")
    unt = par.unit
    if unt is u.Unit():
        unt=""
    col_unit.append(unt)
    col_unit.append(unt)
    
    
t = Table(names=col_names, units=col_unit)

for i in range(axis.nbin):
    col_data = []
    for name in sky_model.parameters.free_parameters.names:
        col_data.append(est.models[i].parameters[name].value)
        col_data.append(est.models[i].parameters[name].error)
    t.add_row(col_data)

In [165]:
t.add_columns(gti.table.columns, indexes=[0,0])

In [14]:
gti

NameError: name 'gti' is not defined

In [114]:
gti = est.axis.to_gti()

In [141]:
import astropy.units as u

In [154]:
sky_model.parameters["index"].unit is u.Unit()

True

In [15]:
gti = GTI.from_time_intervals(time_intervals)

TypeError: object of type 'GTI' has no len()